# Análise de Produtividade de Culturas

Este notebook realiza uma análise completa de dados de produtividade agrícola, incluindo:
- Análise Exploratória de Dados (EDA)
- Análise de Correlação
- Clustering com K-Means
- Identificação de Outliers
- Visualizações 2D e 3D dos Clusters

O objetivo é identificar padrões e insights que possam ajudar a otimizar a produtividade agrícola.

## 1. Configuração e Importação de Bibliotecas

In [ ]:
# Importação das bibliotecas necessárias
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import display, Markdown

# Configurações de visualização
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('viridis')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

# Configuração para exibir todas as colunas do DataFrame
pd.set_option('display.max_columns', None)

# Configuração do diretório de imagens
IMAGES_DIR = os.path.join('..', 'images')
os.makedirs(IMAGES_DIR, exist_ok=True)

## 2. Carregamento e Exploração dos Dados

In [ ]:
def load_data(file_path):
    """
    Carrega os dados do arquivo CSV e retorna um DataFrame pandas.
    
    Args:
        file_path (str): Caminho para o arquivo CSV
        
    Returns:
        pandas.DataFrame: DataFrame com os dados carregados
    """
    try:
        df = pd.read_csv(file_path)
        print(f"Dados carregados com sucesso: {file_path}")
        print(f"Dimensões do dataset: {df.shape[0]} linhas x {df.shape[1]} colunas")
        return df
    except Exception as e:
        print(f"Erro ao carregar os dados: {e}")
        return None

# Carregar os dados
file_path = os.path.join('..', 'data', 'crop_yield.csv')
df = load_data(file_path)

# Exibir as primeiras linhas do DataFrame
display(Markdown("### Primeiras linhas do dataset:"))
display(df.head())

### 2.1 Informações do Dataset

In [ ]:
def display_dataset_info(df):
    """
    Exibe informações sobre o dataset.
    
    Args:
        df (pandas.DataFrame): DataFrame a ser analisado
    """
    display(Markdown("### Informações do Dataset:"))
    display(df.info())
    
    display(Markdown("### Estatísticas Descritivas:"))
    display(df.describe())
    
    display(Markdown("### Tipos de Dados:"))
    display(pd.DataFrame(df.dtypes, columns=['Tipo de Dado']))
    
    # Verificar valores únicos para colunas categóricas
    categorical_cols = df.select_dtypes(include=['object']).columns
    if len(categorical_cols) > 0:
        display(Markdown("### Valores Únicos em Colunas Categóricas:"))
        for col in categorical_cols:
            print(f"\n{col}: {df[col].nunique()} valores únicos")
            display(df[col].value_counts())

# Exibir informações do dataset
display_dataset_info(df)

### 2.2 Análise de Valores Ausentes

In [ ]:
def analyze_missing_values(df):
    """
    Analisa valores ausentes no DataFrame.
    
    Args:
        df (pandas.DataFrame): DataFrame a ser analisado
    """
    display(Markdown("### Análise de Valores Ausentes:"))
    
    # Contagem de valores ausentes por coluna
    missing_values = df.isnull().sum()
    missing_percent = (missing_values / len(df)) * 100
    
    missing_df = pd.DataFrame({
        'Valores Ausentes': missing_values,
        'Percentual (%)': missing_percent
    })
    
    display(missing_df)
    
    # Visualizar valores ausentes
    plt.figure(figsize=(12, 6))
    sns.heatmap(df.isnull(), cbar=False, yticklabels=False, cmap='viridis')
    plt.title('Mapa de Valores Ausentes')
    plt.tight_layout()
    plt.savefig(os.path.join(IMAGES_DIR, 'missing_values_heatmap.png'))
    plt.show()
    
    # Se existirem valores ausentes, sugerir estratégias de tratamento
    if missing_values.sum() > 0:
        display(Markdown("### Estratégias de Tratamento de Valores Ausentes:"))
        for col in df.columns[missing_values > 0]:
            print(f"\nColuna: {col} - {missing_percent[col]:.2f}% ausentes")
            if missing_percent[col] < 5:
                print("  Recomendação: Remover linhas ou imputar com média/mediana")
            elif missing_percent[col] < 30:
                print("  Recomendação: Imputar valores usando técnicas avançadas (KNN, modelos preditivos)")
            else:
                print("  Recomendação: Considerar remover a coluna ou investigar a causa dos valores ausentes")

# Analisar valores ausentes
analyze_missing_values(df)

### 2.3 Tratamento de Valores Ausentes (se necessário)

In [ ]:
def handle_missing_values(df):
    """
    Trata valores ausentes no DataFrame.
    
    Args:
        df (pandas.DataFrame): DataFrame com valores ausentes
        
    Returns:
        pandas.DataFrame: DataFrame com valores ausentes tratados
    """
    # Verificar se existem valores ausentes
    if df.isnull().sum().sum() == 0:
        print("Não há valores ausentes para tratar.")
        return df
    
    # Criar uma cópia do DataFrame para não modificar o original
    df_clean = df.copy()
    
    # Para cada coluna com valores ausentes
    for col in df.columns[df.isnull().sum() > 0]:
        # Se for coluna numérica, imputar com a mediana
        if df[col].dtype in ['int64', 'float64']:
            median_value = df[col].median()
            df_clean[col].fillna(median_value, inplace=True)
            print(f"Valores ausentes em '{col}' preenchidos com a mediana: {median_value}")
        # Se for coluna categórica, imputar com o valor mais frequente
        else:
            mode_value = df[col].mode()[0]
            df_clean[col].fillna(mode_value, inplace=True)
            print(f"Valores ausentes em '{col}' preenchidos com o valor mais frequente: {mode_value}")
    
    # Verificar se todos os valores ausentes foram tratados
    if df_clean.isnull().sum().sum() == 0:
        print("\nTodos os valores ausentes foram tratados com sucesso!")
    else:
        print("\nAtenção: Ainda existem valores ausentes no DataFrame.")
    
    return df_clean

# Tratar valores ausentes
df_clean = handle_missing_values(df)

# Verificar se ainda existem valores ausentes
print(f"\nValores ausentes restantes: {df_clean.isnull().sum().sum()}")

## 3. Análise Exploratória de Dados (EDA)

### 3.1 Análise da Distribuição dos Dados

In [ ]:
def analyze_data_distribution(df):
    """
    Analisa a distribuição das variáveis numéricas no DataFrame.
    
    Args:
        df (pandas.DataFrame): DataFrame a ser analisado
    """
    display(Markdown("### Distribuição das Variáveis Numéricas:"))
    
    # Selecionar apenas colunas numéricas
    numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
    
    # Criar histogramas para cada variável numérica
    n_cols = 2  # Número de colunas no grid de plots
    n_rows = (len(numeric_cols) + n_cols - 1) // n_cols  # Número de linhas necessárias
    
    plt.figure(figsize=(15, n_rows * 5))
    
    for i, col in enumerate(numeric_cols):
        plt.subplot(n_rows, n_cols, i + 1)
        
        # Histograma com KDE
        sns.histplot(df[col], kde=True, color='skyblue')
        
        # Adicionar linha vertical para média e mediana
        plt.axvline(df[col].mean(), color='red', linestyle='--', label=f'Média: {df[col].mean():.2f}')
        plt.axvline(df[col].median(), color='green', linestyle='-.', label=f'Mediana: {df[col].median():.2f}')
        
        plt.title(f'Distribuição de {col}')
        plt.xlabel(col)
        plt.ylabel('Frequência')
        plt.legend()
        plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(os.path.join(IMAGES_DIR, 'numeric_distributions.png'))
    plt.show()
    
    # Criar boxplots para identificar outliers
    plt.figure(figsize=(15, n_rows * 4))
    
    for i, col in enumerate(numeric_cols):
        plt.subplot(n_rows, n_cols, i + 1)
        
        # Boxplot
        sns.boxplot(x=df[col], color='skyblue')
        
        plt.title(f'Boxplot de {col}')
        plt.xlabel(col)
        plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(os.path.join(IMAGES_DIR, 'numeric_boxplots.png'))
    plt.show()
    
    # Se houver variáveis categóricas, criar gráficos de contagem
    categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
    
    if len(categorical_cols) > 0:
        display(Markdown("### Distribuição das Variáveis Categóricas:"))
        
        n_rows_cat = (len(categorical_cols) + n_cols - 1) // n_cols
        plt.figure(figsize=(15, n_rows_cat * 5))
        
        for i, col in enumerate(categorical_cols):
            plt.subplot(n_rows_cat, n_cols, i + 1)
            
            # Gráfico de contagem
            value_counts = df[col].value_counts().sort_values(ascending=False)
            sns.barplot(x=value_counts.index, y=value_counts.values, palette='viridis')
            
            plt.title(f'Contagem de {col}')
            plt.xlabel(col)
            plt.ylabel('Contagem')
            plt.xticks(rotation=45)
            plt.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig(os.path.join(IMAGES_DIR, 'categorical_distributions.png'))
        plt.show()

# Analisar distribuição dos dados
analyze_data_distribution(df_clean)

### 3.2 Análise de Correlação

In [ ]:
def analyze_correlations(df):
    """
    Analisa correlações entre variáveis numéricas no DataFrame.
    
    Args:
        df (pandas.DataFrame): DataFrame a ser analisado
    """
    display(Markdown("### Matriz de Correlação:"))
    
    # Selecionar apenas colunas numéricas
    numeric_df = df.select_dtypes(include=['int64', 'float64'])
    
    # Calcular matriz de correlação
    corr_matrix = numeric_df.corr()
    
    # Exibir matriz de correlação
    display(corr_matrix)
    
    # Visualizar matriz de correlação como heatmap
    plt.figure(figsize=(12, 10))
    mask = np.triu(np.ones_like(corr_matrix, dtype=bool))  # Máscara para exibir apenas o triângulo inferior
    cmap = sns.diverging_palette(230, 20, as_cmap=True)
    
    sns.heatmap(corr_matrix, mask=mask, cmap=cmap, vmax=1, vmin=-1, center=0,
                square=True, linewidths=.5, annot=True, fmt='.2f', cbar_kws={"shrink": .5})
    
    plt.title('Matriz de Correlação')
    plt.tight_layout()
    plt.savefig(os.path.join(IMAGES_DIR, 'correlation_matrix.png'))
    plt.show()
    
    # Identificar correlações fortes (positivas e negativas)
    display(Markdown("### Correlações Significativas:"))
    
    # Converter a matriz de correlação em um DataFrame de pares
    corr_pairs = corr_matrix.unstack().reset_index()
    corr_pairs.columns = ['Variável 1', 'Variável 2', 'Correlação']
    
    # Remover autocorrelações e duplicatas
    corr_pairs = corr_pairs[corr_pairs['Variável 1'] != corr_pairs['Variável 2']]
    corr_pairs = corr_pairs.drop_duplicates(['Correlação'])
    
    # Ordenar por valor absoluto da correlação (decrescente)
    corr_pairs['Abs_Correlação'] = corr_pairs['Correlação'].abs()
    corr_pairs = corr_pairs.sort_values('Abs_Correlação', ascending=False).drop('Abs_Correlação', axis=1)
    
    # Exibir correlações significativas (|r| > 0.5)
    significant_corr = corr_pairs[corr_pairs['Correlação'].abs() > 0.5]
    
    if len(significant_corr) > 0:
        display(significant_corr)
        
        # Visualizar pares com correlações mais fortes
        display(Markdown("### Visualização de Pares com Correlações Significativas:"))
        
        # Limitar a 5 pares mais correlacionados para não sobrecarregar
        top_pairs = significant_corr.head(5)
        
        for _, row in top_pairs.iterrows():
            var1, var2, corr = row['Variável 1'], row['Variável 2'], row['Correlação']
            
            plt.figure(figsize=(10, 6))
            sns.scatterplot(x=df[var1], y=df[var2], alpha=0.7)
            
            # Adicionar linha de tendência
            sns.regplot(x=df[var1], y=df[var2], scatter=False, color='red')
            
            plt.title(f'Correlação entre {var1} e {var2}: {corr:.2f}')
            plt.xlabel(var1)
            plt.ylabel(var2)
            plt.grid(True, alpha=0.3)
            plt.savefig(os.path.join(IMAGES_DIR, f'correlation_{var1}_{var2}.png'))
            plt.show()
    else:
        print("Não foram encontradas correlações significativas (|r| > 0.5) entre as variáveis.")

# Analisar correlações
analyze_correlations(df_clean)

### 3.3 Visualização de Pares de Variáveis

In [ ]:
# Visualizar pares de variáveis numéricas
numeric_cols = df_clean.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Se houver muitas colunas numéricas, selecionar apenas as mais relevantes
if len(numeric_cols) > 5:
    # Selecionar as colunas mais relevantes (exemplo: as primeiras 5)
    selected_cols = numeric_cols[:5]
    print(f"Selecionando apenas {len(selected_cols)} colunas para visualização de pares: {selected_cols}")
else:
    selected_cols = numeric_cols

# Criar matriz de gráficos de dispersão
plt.figure(figsize=(15, 15))
sns.pairplot(df_clean[selected_cols], diag_kind='kde', plot_kws={'alpha': 0.6})
plt.suptitle('Matriz de Gráficos de Dispersão', y=1.02, fontsize=16)
plt.savefig(os.path.join(IMAGES_DIR, 'pairplot.png'))
plt.show()

## 4. Análise de Clustering

### 4.1 Pré-processamento de Dados para Clustering

In [ ]:
def preprocess_data_for_clustering(df):
    """
    Prepara os dados para análise de clustering.
    
    Args:
        df (pandas.DataFrame): DataFrame a ser processado
        
    Returns:
        tuple: (DataFrame com dados processados, array com features escaladas, lista de nomes de features, objeto scaler)
    """
    print("\n" + "="*50)
    print("PRÉ-PROCESSAMENTO DE DADOS PARA CLUSTERING")
    print("="*50)
    
    # Criar uma cópia do DataFrame para não modificar o original
    cluster_df = df.copy()
    
    # Selecionar apenas colunas numéricas para clustering
    numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
    
    # Se 'Yield' estiver nas colunas, removê-la das features (será nossa variável alvo)
    feature_names = [col for col in numeric_cols if col.lower() != 'yield']
    
    print(f"Features selecionadas para clustering: {feature_names}")
    
    # Escalar as features para garantir que todas tenham o mesmo peso
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(cluster_df[feature_names])
    
    print(f"Dados escalados com StandardScaler. Shape: {scaled_features.shape}")
    
    return cluster_df, scaled_features, feature_names, scaler

# Pré-processar dados para clustering
cluster_df, scaled_features, feature_names, scaler = preprocess_data_for_clustering(df_clean)

### 4.2 Determinação do Número Ótimo de Clusters

In [ ]:
def determine_optimal_clusters(scaled_features, max_clusters=10):
    """
    Determina o número ótimo de clusters usando o método do cotovelo e score de silhueta.
    
    Args:
        scaled_features (numpy.ndarray): Features escaladas
        max_clusters (int): Número máximo de clusters a testar
        
    Returns:
        int: Número ótimo de clusters
    """
    print("\n" + "="*50)
    print("DETERMINAÇÃO DO NÚMERO ÓTIMO DE CLUSTERS")
    print("="*50)
    
    # Calcular a inércia (soma dos quadrados das distâncias) para diferentes valores de k
    inertia = []
    silhouette_scores = []
    k_values = range(2, max_clusters + 1)
    
    for k in k_values:
        kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
        kmeans.fit(scaled_features)
        inertia.append(kmeans.inertia_)
        
        # Calcular score de silhueta
        labels = kmeans.labels_
        silhouette_avg = silhouette_score(scaled_features, labels)
        silhouette_scores.append(silhouette_avg)
        
        print(f"k={k}: Inércia={kmeans.inertia_:.2f}, Score de Silhueta={silhouette_avg:.4f}")
    
    # Visualizar o método do cotovelo
    plt.figure(figsize=(12, 5))
    
    # Plot da inércia (método do cotovelo)
    plt.subplot(1, 2, 1)
    plt.plot(k_values, inertia, 'o-', color='blue')
    plt.xlabel('Número de Clusters (k)')
    plt.ylabel('Inércia')
    plt.title('Método do Cotovelo')
    plt.grid(True, alpha=0.3)
    
    # Plot do score de silhueta
    plt.subplot(1, 2, 2)
    plt.plot(k_values, silhouette_scores, 'o-', color='green')
    plt.xlabel('Número de Clusters (k)')
    plt.ylabel('Score de Silhueta')
    plt.title('Score de Silhueta')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(os.path.join(IMAGES_DIR, 'optimal_clusters.png'))
    plt.show()
    
    # Determinar o número ótimo de clusters
    # Baseado no maior score de silhueta
    optimal_k = k_values[silhouette_scores.index(max(silhouette_scores))]
    
    print(f"\nNúmero ótimo de clusters (baseado no score de silhueta): {optimal_k}")
    
    return optimal_k

# Determinar o número ótimo de clusters
optimal_k = determine_optimal_clusters(scaled_features)

### 4.3 Implementação do Algoritmo K-Means

In [ ]:
def perform_kmeans_clustering(cluster_df, scaled_features, n_clusters, feature_names):
    """
    Realiza o clustering usando o algoritmo K-Means.
    
    Args:
        cluster_df (pandas.DataFrame): DataFrame original
        scaled_features (numpy.ndarray): Features escaladas
        n_clusters (int): Número de clusters
        feature_names (list): Nomes das features usadas para clustering
        
    Returns:
        tuple: (DataFrame com atribuições de cluster, DataFrame com centros dos clusters)
    """
    print("\n" + "="*50)
    print(f"CLUSTERING COM K-MEANS (k={n_clusters})")
    print("="*50)
    
    # Aplicar K-Means com o número ótimo de clusters
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    cluster_labels = kmeans.fit_predict(scaled_features)
    
    # Adicionar labels de cluster ao DataFrame original
    cluster_df = cluster_df.copy()
    cluster_df['Cluster'] = cluster_labels
    
    # Contar o número de amostras em cada cluster
    cluster_counts = cluster_df['Cluster'].value_counts().sort_index()
    for cluster, count in cluster_counts.items():
        print(f"Cluster {cluster}: {count} amostras ({count/len(cluster_df)*100:.1f}%)")
    
    # Obter os centros dos clusters e convertê-los de volta para a escala original
    centers = kmeans.cluster_centers_
    
    # Criar um DataFrame com os centros dos clusters
    centers_df = pd.DataFrame(centers, columns=feature_names)
    centers_df.index.name = 'Cluster'
    
    # Exibir os centros dos clusters
    print("\nCentros dos Clusters (features escaladas):")
    display(centers_df)
    
    return cluster_df, centers_df

# Realizar clustering com K-Means
cluster_df, centers_df = perform_kmeans_clustering(cluster_df, scaled_features, optimal_k, feature_names)

### 4.4 Visualização dos Clusters em 2D

In [ ]:
def visualize_clusters_2d(cluster_df, feature_names):
    """
    Visualiza os clusters em 2D usando as duas primeiras features.
    
    Args:
        cluster_df (pandas.DataFrame): DataFrame com atribuições de cluster
        feature_names (list): Nomes das features usadas para clustering
    """
    print("\n" + "="*50)
    print("VISUALIZAÇÃO DOS CLUSTERS EM 2D")
    print("="*50)
    
    # Verificar se temos pelo menos 2 features para visualização 2D
    if len(feature_names) < 2:
        print("Não há features suficientes para visualização 2D.")
        return
    
    # Selecionar as duas primeiras features para visualização
    feature1, feature2 = feature_names[0], feature_names[1]
    
    plt.figure(figsize=(12, 10))
    
    # Plotar pontos coloridos por cluster
    scatter = plt.scatter(cluster_df[feature1], cluster_df[feature2], 
                c=cluster_df['Cluster'], cmap='viridis', 
                s=50, alpha=0.7, edgecolors='w')
    
    # Adicionar legenda
    legend1 = plt.legend(*scatter.legend_elements(), title="Clusters")
    plt.gca().add_artist(legend1)
    
    # Adicionar rótulos e título
    plt.xlabel(feature1)
    plt.ylabel(feature2)
    plt.title(f'Visualização dos Clusters: {feature1} vs {feature2}')
    plt.grid(True, alpha=0.3)
    
    # Adicionar centros dos clusters
    centers = centers_df[[feature1, feature2]].values
    plt.scatter(centers[:, 0], centers[:, 1], c='red', marker='X', s=200, alpha=1, label='Centros')
    
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(IMAGES_DIR, 'clusters_2d.png'))
    plt.show()
    
    # Se tivermos mais de 2 features, criar visualizações adicionais com diferentes pares
    if len(feature_names) > 2:
        print("\nVisualizando clusters com diferentes pares de features:")
        
        # Selecionar até 3 pares adicionais de features
        pairs = []
        for i in range(min(3, len(feature_names) - 1)):
            for j in range(i + 1, min(i + 2, len(feature_names))):
                if i != 0 or j != 1:  # Evitar repetir o primeiro par
                    pairs.append((feature_names[i], feature_names[j]))
        
        # Plotar cada par adicional
        for idx, (feat1, feat2) in enumerate(pairs):
            plt.figure(figsize=(10, 8))
            
            # Plotar pontos coloridos por cluster
            scatter = plt.scatter(cluster_df[feat1], cluster_df[feat2], 
                        c=cluster_df['Cluster'], cmap='viridis', 
                        s=50, alpha=0.7, edgecolors='w')
            
            # Adicionar legenda
            legend1 = plt.legend(*scatter.legend_elements(), title="Clusters")
            plt.gca().add_artist(legend1)
            
            # Adicionar rótulos e título
            plt.xlabel(feat1)
            plt.ylabel(feat2)
            plt.title(f'Visualização dos Clusters: {feat1} vs {feat2}')
            plt.grid(True, alpha=0.3)
            
            # Adicionar centros dos clusters
            centers = centers_df[[feat1, feat2]].values
            plt.scatter(centers[:, 0], centers[:, 1], c='red', marker='X', s=200, alpha=1, label='Centros')
            
            plt.legend()
            plt.tight_layout()
            plt.savefig(os.path.join(IMAGES_DIR, f'clusters_2d_{idx+1}.png'))
            plt.show()

# Visualizar clusters em 2D
visualize_clusters_2d(cluster_df, feature_names)

### 4.5 Visualização dos Clusters em 3D

In [ ]:
def visualize_clusters_3d(cluster_df, scaled_features):
    """
    Visualiza os clusters em 3D se houver pelo menos 3 features.
    
    Args:
        cluster_df (pandas.DataFrame): DataFrame com atribuições de cluster
        scaled_features (numpy.ndarray): Features escaladas
    """
    print("\n" + "="*50)
    print("VISUALIZAÇÃO DOS CLUSTERS EM 3D")
    print("="*50)
    
    # Verificar se temos pelo menos 3 features para visualização 3D
    if scaled_features.shape[1] < 3:
        print("Não há features suficientes para visualização 3D. São necessárias pelo menos 3 features.")
        return
    
    # Selecionar as três primeiras features para visualização 3D
    feature1, feature2, feature3 = feature_names[0], feature_names[1], feature_names[2]
    
    # Criar figura 3D
    fig = plt.figure(figsize=(12, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    # Plotar pontos 3D coloridos por cluster
    scatter = ax.scatter(
        cluster_df[feature1], 
        cluster_df[feature2], 
        cluster_df[feature3],
        c=cluster_df['Cluster'], 
        cmap='viridis',
        s=50, 
        alpha=0.7
    )
    
    # Adicionar centros dos clusters
    centers = centers_df[[feature1, feature2, feature3]].values
    ax.scatter(
        centers[:, 0], 
        centers[:, 1], 
        centers[:, 2],
        c='red', 
        marker='X', 
        s=200, 
        alpha=1, 
        label='Centros'
    )
    
    # Adicionar rótulos e título
    ax.set_xlabel(feature1)
    ax.set_ylabel(feature2)
    ax.set_zlabel(feature3)
    ax.set_title(f'Visualização 3D dos Clusters')
    
    # Adicionar legenda
    legend1 = ax.legend(*scatter.legend_elements(), title="Clusters")
    ax.add_artist(legend1)
    
    # Adicionar legenda para os centros
    ax.legend(["Centros"], loc="upper right")
    
    plt.tight_layout()
    plt.savefig(os.path.join(IMAGES_DIR, 'clusters_3d.png'))
    plt.show()

# Visualizar clusters em 3D
visualize_clusters_3d(cluster_df, scaled_features)

### 4.6 Identificação de Outliers dentro dos Clusters

In [ ]:
def identify_cluster_outliers(cluster_df, feature_names):
    """
    Identifica outliers dentro de cada cluster usando métodos estatísticos.
    
    Args:
        cluster_df (pandas.DataFrame): DataFrame com atribuições de cluster
        feature_names (list): Nomes das features usadas para clustering
        
    Returns:
        pandas.DataFrame: DataFrame com flags de outliers
    """
    print("\n" + "="*50)
    print("IDENTIFICANDO OUTLIERS DENTRO DOS CLUSTERS")
    print("="*50)
    
    # Criar uma cópia do DataFrame para adicionar informações de outliers
    outlier_df = cluster_df.copy()
    outlier_df['is_outlier'] = False
    
    # Para cada cluster, identificar outliers usando o método IQR
    for cluster in sorted(outlier_df['Cluster'].unique()):
        cluster_data = outlier_df[outlier_df['Cluster'] == cluster]
        
        # Calcular outliers para cada feature
        for feature in feature_names:
            Q1 = cluster_data[feature].quantile(0.25)
            Q3 = cluster_data[feature].quantile(0.75)
            IQR = Q3 - Q1
            
            # Definir limites para outliers (1.5 * IQR)
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            
            # Marcar outliers
            feature_outliers = (cluster_data[feature] < lower_bound) | (cluster_data[feature] > upper_bound)
            outlier_indices = cluster_data[feature_outliers].index
            outlier_df.loc[outlier_indices, 'is_outlier'] = True
    
    # Contar outliers
    outlier_count = outlier_df['is_outlier'].sum()
    print(f"Total de outliers identificados: {outlier_count} ({outlier_count/len(outlier_df)*100:.1f}% dos dados)")
    
    # Resumir outliers por cluster
    print("\nOutliers por cluster:")
    for cluster in sorted(outlier_df['Cluster'].unique()):
        cluster_data = outlier_df[outlier_df['Cluster'] == cluster]
        cluster_outliers = cluster_data['is_outlier'].sum()
        print(f"Cluster {cluster}: {cluster_outliers} outliers ({cluster_outliers/len(cluster_data)*100:.1f}% do cluster)")
    
    # Visualizar outliers em 2D
    if len(feature_names) >= 2:
        plt.figure(figsize=(12, 10))
        
        # Plotar pontos normais
        non_outliers = outlier_df[~outlier_df['is_outlier']]
        plt.scatter(non_outliers[feature_names[0]], non_outliers[feature_names[1]], 
                    c=non_outliers['Cluster'], cmap='viridis', 
                    s=50, alpha=0.7, edgecolors='w', label='Normal')
        
        # Plotar outliers
        outliers = outlier_df[outlier_df['is_outlier']]
        plt.scatter(outliers[feature_names[0]], outliers[feature_names[1]], 
                    c=outliers['Cluster'], cmap='viridis', 
                    s=100, alpha=1.0, edgecolors='red', linewidth=2, marker='X', label='Outlier')
        
        plt.xlabel(feature_names[0])
        plt.ylabel(feature_names[1])
        plt.title('Outliers nos Clusters')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.savefig(os.path.join(IMAGES_DIR, 'cluster_outliers.png'))
        plt.show()
        print(f"Visualização de outliers nos clusters salva em '{os.path.join(IMAGES_DIR, 'cluster_outliers.png')}'")
    
    # Analisar características dos outliers
    if outlier_count > 0:
        print("\nCaracterísticas dos outliers:")
        outliers = outlier_df[outlier_df['is_outlier']]
        non_outliers = outlier_df[~outlier_df['is_outlier']]
        
        for feature in feature_names:
            outlier_mean = outliers[feature].mean()
            non_outlier_mean = non_outliers[feature].mean()
            print(f"{feature}: Média dos outliers = {outlier_mean:.2f}, Média dos não-outliers = {non_outlier_mean:.2f}, Diferença = {outlier_mean - non_outlier_mean:.2f}")
    
    return outlier_df

# Identificar outliers dentro dos clusters
outlier_df = identify_cluster_outliers(cluster_df, feature_names)

### 4.7 Resumo dos Resultados do Clustering

In [ ]:
def summarize_clustering_results(cluster_df, centers_df, feature_names, outlier_df=None):
    """
    Resumo dos resultados do clustering, incluindo características dos clusters e insights.
    
    Args:
        cluster_df (pandas.DataFrame): DataFrame com atribuições de cluster
        centers_df (pandas.DataFrame): DataFrame com centros dos clusters
        feature_names (list): Nomes das features usadas para clustering
        outlier_df (pandas.DataFrame, optional): DataFrame com flags de outliers
    """
    print("\n" + "="*50)
    print("RESUMO DOS RESULTADOS DO CLUSTERING")
    print("="*50)
    
    # 1. Informações gerais
    n_clusters = len(centers_df)
    print(f"Número de clusters: {n_clusters}")
    print(f"Número de amostras: {len(cluster_df)}")
    print(f"Features utilizadas: {', '.join(feature_names)}")
    
    # 2. Distribuição de amostras por cluster
    print("\nDistribuição de amostras por cluster:")
    cluster_counts = cluster_df['Cluster'].value_counts().sort_index()
    
    # Criar gráfico de barras para a distribuição de clusters
    plt.figure(figsize=(10, 6))
    bars = plt.bar(cluster_counts.index, cluster_counts.values, color='skyblue')
    
    # Adicionar rótulos nas barras
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height + 0.1,
                 f'{height} ({height/len(cluster_df)*100:.1f}%)',
                 ha='center', va='bottom')
    
    plt.xlabel('Cluster')
    plt.ylabel('Número de Amostras')
    plt.title('Distribuição de Amostras por Cluster')
    plt.xticks(cluster_counts.index)
    plt.grid(axis='y', alpha=0.3)
    plt.tight_layout()
    plt.savefig(os.path.join(IMAGES_DIR, 'cluster_distribution.png'))
    plt.show()
    
    # 3. Características de cada cluster
    print("\nCaracterísticas de cada cluster (médias):")
    
    # Calcular médias para cada feature em cada cluster
    cluster_means = cluster_df.groupby('Cluster')[feature_names].mean()
    
    # Exibir tabela de médias
    display(cluster_means)
    
    # 4. Visualização de radar chart para comparar clusters
    print("\nComparando características dos clusters (Radar Chart):")
    
    # Normalizar os dados para o radar chart (0-1)
    scaler = MinMaxScaler()
    cluster_means_scaled = pd.DataFrame(
        scaler.fit_transform(cluster_means),
        columns=cluster_means.columns,
        index=cluster_means.index
    )
    
    # Configurar o radar chart
    categories = feature_names
    N = len(categories)
    
    # Calcular os ângulos para cada eixo
    angles = [n / float(N) * 2 * np.pi for n in range(N)]
    angles += angles[:1]  # Fechar o círculo
    
    # Criar figura
    fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(polar=True))
    
    # Adicionar cada cluster ao radar chart
    for cluster in cluster_means_scaled.index:
        values = cluster_means_scaled.loc[cluster].values.tolist()
        values += values[:1]  # Fechar o círculo
        
        # Plotar os valores
        ax.plot(angles, values, linewidth=2, linestyle='solid', label=f'Cluster {cluster}')
        ax.fill(angles, values, alpha=0.1)
    
    # Configurar o radar chart
    plt.xticks(angles[:-1], categories)
    ax.set_rlabel_position(0)
    plt.yticks([0, 0.25, 0.5, 0.75, 1], ['0', '0.25', '0.5', '0.75', '1'], color='grey', size=7)
    plt.ylim(0, 1)
    
    # Adicionar legenda e título
    plt.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1))
    plt.title('Comparação de Características entre Clusters', size=15, y=1.1)
    plt.tight_layout()
    plt.savefig(os.path.join(IMAGES_DIR, 'cluster_radar_chart.png'))
    plt.show()
    
    # 5. Insights e conclusões
    print("\nInsights e Conclusões:")
    
    # Identificar o cluster com maior produtividade (yield)
    if 'yield' in feature_names:
        best_cluster = cluster_means['yield'].idxmax()
        print(f"- O Cluster {best_cluster} apresenta a maior produtividade média.")
        
        # Identificar as características do melhor cluster
        print("  Características deste cluster:")
        for feature in feature_names:
            if feature != 'yield':
                value = cluster_means.loc[best_cluster, feature]
                overall_mean = cluster_df[feature].mean()
                diff = ((value - overall_mean) / overall_mean) * 100
                direction = 'maior' if diff > 0 else 'menor'
                print(f"  - {feature}: {value:.2f} ({abs(diff):.1f}% {direction} que a média geral)")
    
    # Verificar relações entre clusters e outliers
    if outlier_df is not None and 'is_outlier' in outlier_df.columns:
        print("\nRelação entre Clusters e Outliers:")
        outlier_by_cluster = outlier_df.groupby('Cluster')['is_outlier'].mean() * 100
        print(f"- Percentual de outliers por cluster:")
        for cluster, pct in outlier_by_cluster.items():
            print(f"  Cluster {cluster}: {pct:.1f}% de outliers")
        
        # Identificar o cluster com maior percentual de outliers
        if len(outlier_by_cluster) > 0:
            most_outliers_cluster = outlier_by_cluster.idxmax()
            print(f"- O Cluster {most_outliers_cluster} possui o maior percentual de outliers ({outlier_by_cluster[most_outliers_cluster]:.1f}%).")
            print("  Isso pode indicar maior variabilidade ou presença de casos atípicos neste grupo.")
    
    # 6. Sugestões para análise adicional
    print("\nSugestões para Análise Adicional:")
    print("- Realizar análise de regressão para cada cluster separadamente para identificar fatores específicos que afetam a produtividade em cada grupo.")
    print("- Investigar as condições ambientais e práticas agrícolas associadas a cada cluster para entender melhor as diferenças.")
    print("- Considerar a aplicação de técnicas de aprendizado supervisionado para prever a produtividade com base nas características identificadas.")
    print("- Explorar a possibilidade de criar recomendações personalizadas para cada grupo de produtores com base nas características do cluster.")

# Resumir os resultados do clustering
summarize_clustering_results(cluster_df, centers_df, feature_names, outlier_df)

## 5. Conclusões da Análise de Clustering

Nesta análise, aplicamos o algoritmo K-Means para identificar padrões naturais nos dados de produtividade agrícola. Através da análise de clustering, conseguimos identificar grupos distintos de produtores/regiões com características semelhantes.

### Principais Descobertas:

1. **Identificação de Grupos Distintos**: O algoritmo K-Means identificou grupos com características distintas em termos de variáveis como temperatura, umidade, precipitação e produtividade.

2. **Relação entre Variáveis e Produtividade**: Observamos como diferentes combinações de variáveis ambientais estão associadas a níveis distintos de produtividade.

3. **Identificação de Outliers**: A análise permitiu identificar casos atípicos dentro de cada cluster, que podem representar condições extremas ou erros de medição.

### Implicações Práticas:

- **Recomendações Personalizadas**: Com base nos clusters identificados, podemos desenvolver recomendações específicas para cada grupo de produtores.

- **Otimização de Recursos**: Entender as características de cada cluster permite alocar recursos de forma mais eficiente, focando em intervenções específicas para cada grupo.

- **Previsão de Produtividade**: Os padrões identificados podem ser utilizados para desenvolver modelos preditivos mais precisos para cada grupo.

### Próximos Passos:

Na próxima etapa deste projeto, iremos desenvolver modelos de aprendizado supervisionado para prever a produtividade com base nas variáveis analisadas. Utilizaremos os insights obtidos na análise de clustering para informar o desenvolvimento desses modelos preditivos.

## 6. Modelagem Preditiva (Aprendizado Supervisionado)

Nesta seção, iremos desenvolver modelos de aprendizado supervisionado para prever a produtividade das culturas com base nas variáveis ambientais e agrícolas. Vamos implementar e comparar cinco modelos diferentes:

1. **Regressão Linear**
2. **Árvore de Decisão**
3. **Random Forest**
4. **Gradient Boosting (XGBoost)**
5. **Rede Neural (MLP Regressor)**

Para cada modelo, vamos avaliar seu desempenho usando métricas como R², MAE, MSE e RMSE, e comparar os resultados para identificar o modelo mais adequado para nosso conjunto de dados.

### 6.1 Preparação dos Dados para Modelagem

In [ ]:
def prepare_data_for_modeling(df, target_column='yield', test_size=0.2, random_state=42):
    """
    Prepara os dados para modelagem preditiva, separando features e target,
    e dividindo em conjuntos de treino e teste.
    
    Args:
        df (pandas.DataFrame): DataFrame com os dados
        target_column (str): Nome da coluna alvo (produtividade)
        test_size (float): Proporção dos dados para teste (0.0-1.0)
        random_state (int): Semente aleatória para reprodutibilidade
        
    Returns:
        tuple: (X_train, X_test, y_train, y_test, feature_names)
    """
    print("\n" + "="*50)
    print("PREPARAÇÃO DOS DADOS PARA MODELAGEM PREDITIVA")
    print("="*50)
    
    # Verificar se a coluna alvo existe no DataFrame
    if target_column not in df.columns:
        raise ValueError(f"A coluna alvo '{target_column}' não foi encontrada no DataFrame.")
    
    # Remover colunas que não devem ser usadas como features
    # Neste caso, se tivermos a coluna 'Cluster' da análise anterior, vamos removê-la
    columns_to_drop = ['Cluster', 'is_outlier'] if 'Cluster' in df.columns else []
    
    # Separar features e target
    X = df.drop([target_column] + columns_to_drop, axis=1)
    y = df[target_column]
    
    # Guardar nomes das features para uso posterior
    feature_names = X.columns.tolist()
    
    print(f"Número total de amostras: {len(df)}")
    print(f"Número de features: {len(feature_names)}")
    print(f"Features utilizadas: {', '.join(feature_names)}")
    print(f"Variável alvo: {target_column}")
    
    # Dividir os dados em conjuntos de treino e teste (80% treino, 20% teste)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state
    )
    
    print(f"Tamanho do conjunto de treino: {len(X_train)} amostras ({(1-test_size)*100:.0f}%)")
    print(f"Tamanho do conjunto de teste: {len(X_test)} amostras ({test_size*100:.0f}%)")
    
    # Normalizar os dados
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    print("\nDados normalizados com StandardScaler.")
    
    # Verificar distribuição da variável alvo
    plt.figure(figsize=(10, 6))
    plt.hist(y, bins=20, color='skyblue', edgecolor='black')
    plt.title(f'Distribuição da Variável Alvo ({target_column})')
    plt.xlabel(target_column)
    plt.ylabel('Frequência')
    plt.grid(alpha=0.3)
    plt.savefig(os.path.join(IMAGES_DIR, 'target_distribution.png'))
    plt.show()
    
    return X_train_scaled, X_test_scaled, y_train, y_test, feature_names, scaler

# Preparar dados para modelagem
X_train, X_test, y_train, y_test, model_features, scaler = prepare_data_for_modeling(df)

### 6.2 Funções de Avaliação de Modelos

In [ ]:
def evaluate_regression_model(model, X_train, X_test, y_train, y_test, model_name):
    """
    Avalia um modelo de regressão usando várias métricas.
    
    Args:
        model: Modelo de regressão treinado
        X_train: Features de treino
        X_test: Features de teste
        y_train: Target de treino
        y_test: Target de teste
        model_name (str): Nome do modelo para exibição
        
    Returns:
        dict: Dicionário com as métricas de avaliação
    """
    # Fazer predições nos conjuntos de treino e teste
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Calcular métricas de avaliação
    train_r2 = r2_score(y_train, y_train_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    
    train_mae = mean_absolute_error(y_train, y_train_pred)
    test_mae = mean_absolute_error(y_test, y_test_pred)
    
    train_mse = mean_squared_error(y_train, y_train_pred)
    test_mse = mean_squared_error(y_test, y_test_pred)
    
    train_rmse = np.sqrt(train_mse)
    test_rmse = np.sqrt(test_mse)
    
    # Exibir resultados
    print(f"\n{'-'*20} {model_name} {'-'*20}")
    print(f"R² Score (Treino): {train_r2:.4f}")
    print(f"R² Score (Teste): {test_r2:.4f}")
    print(f"MAE (Treino): {train_mae:.4f}")
    print(f"MAE (Teste): {test_mae:.4f}")
    print(f"MSE (Treino): {train_mse:.4f}")
    print(f"MSE (Teste): {test_mse:.4f}")
    print(f"RMSE (Treino): {train_rmse:.4f}")
    print(f"RMSE (Teste): {test_rmse:.4f}")
    
    # Plotar valores reais vs. previsões
    plt.figure(figsize=(10, 6))
    plt.scatter(y_test, y_test_pred, alpha=0.7)
    
    # Adicionar linha de referência (predições perfeitas)
    min_val = min(min(y_test), min(y_test_pred))
    max_val = max(max(y_test), max(y_test_pred))
    plt.plot([min_val, max_val], [min_val, max_val], 'r--')
    
    plt.xlabel('Valores Reais')
    plt.ylabel('Valores Previstos')
    plt.title(f'{model_name}: Valores Reais vs. Previstos')
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.savefig(os.path.join(IMAGES_DIR, f'{model_name.lower().replace(" ", "_")}_predictions.png'))
    plt.show()
    
    # Retornar métricas em um dicionário
    metrics = {
        'model_name': model_name,
        'train_r2': train_r2,
        'test_r2': test_r2,
        'train_mae': train_mae,
        'test_mae': test_mae,
        'train_mse': train_mse,
        'test_mse': test_mse,
        'train_rmse': train_rmse,
        'test_rmse': test_rmse
    }
    
    return metrics

def plot_feature_importance(model, feature_names, model_name):
    """
    Plota a importância das features para modelos que suportam esta funcionalidade.
    
    Args:
        model: Modelo treinado
        feature_names (list): Lista com nomes das features
        model_name (str): Nome do modelo para exibição
    """
    # Verificar se o modelo suporta feature_importance
    if hasattr(model, 'feature_importances_'):
        # Obter importância das features
        importances = model.feature_importances_
        
        # Criar DataFrame para facilitar a ordenação
        feature_importance_df = pd.DataFrame({
            'Feature': feature_names,
            'Importance': importances
        })
        
        # Ordenar por importância
        feature_importance_df = feature_importance_df.sort_values('Importance', ascending=False)
        
        # Plotar
        plt.figure(figsize=(10, 6))
        plt.barh(feature_importance_df['Feature'], feature_importance_df['Importance'])
        plt.xlabel('Importância')
        plt.ylabel('Feature')
        plt.title(f'Importância das Features - {model_name}')
        plt.tight_layout()
        plt.savefig(os.path.join(IMAGES_DIR, f'{model_name.lower().replace(" ", "_")}_feature_importance.png'))
        plt.show()
        
        return feature_importance_df
    elif hasattr(model, 'coef_'):
        # Para modelos lineares
        coefficients = model.coef_
        
        # Criar DataFrame para facilitar a ordenação
        feature_importance_df = pd.DataFrame({
            'Feature': feature_names,
            'Coefficient': np.abs(coefficients)  # Usar valor absoluto para importância
        })
        
        # Ordenar por importância (valor absoluto do coeficiente)
        feature_importance_df = feature_importance_df.sort_values('Coefficient', ascending=False)
        
        # Plotar
        plt.figure(figsize=(10, 6))
        plt.barh(feature_importance_df['Feature'], feature_importance_df['Coefficient'])
        plt.xlabel('Importância (|Coeficiente|)')
        plt.ylabel('Feature')
        plt.title(f'Importância das Features - {model_name}')
        plt.tight_layout()
        plt.savefig(os.path.join(IMAGES_DIR, f'{model_name.lower().replace(" ", "_")}_feature_importance.png'))
        plt.show()
        
        return feature_importance_df
    else:
        print(f"O modelo {model_name} não suporta visualização de importância de features.")
        return None

### 6.1 Preparação dos Dados para Modelagem

In [ ]:
# Preparar os dados para modelagem preditiva
print("="*50)
print("PREPARAÇÃO DOS DADOS PARA MODELAGEM PREDITIVA")
print("="*50)

# Definir a variável alvo (target) e as features
target_column = 'yield'  # Produtividade como variável alvo

# Selecionar as features relevantes com base na análise exploratória
# Excluir colunas que não são relevantes para a predição
feature_columns = [col for col in df.columns if col != target_column]

# Criar conjuntos de dados para features e target
X = df[feature_columns]
y = df[target_column]

# Exibir informações sobre os dados
print(f"\nDimensões do conjunto de features (X): {X.shape}")
print(f"Dimensões do conjunto alvo (y): {y.shape}")
print(f"Features utilizadas: {', '.join(feature_columns)}")

# Dividir os dados em conjuntos de treino e teste (80% treino, 20% teste)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"\nTamanho do conjunto de treino: {X_train.shape[0]} amostras")
print(f"Tamanho do conjunto de teste: {X_test.shape[0]} amostras")

# Normalizar os dados (opcional, mas recomendado para alguns modelos)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Converter de volta para DataFrame para manter os nomes das colunas
X_train = pd.DataFrame(X_train_scaled, columns=feature_columns)
X_test = pd.DataFrame(X_test_scaled, columns=feature_columns)

# Armazenar os nomes das features para uso posterior
model_features = feature_columns

print("\nDados preparados com sucesso para modelagem preditiva!")

### 6.2 Funções para Avaliação de Modelos

In [ ]:
def evaluate_regression_model(model, X_train, X_test, y_train, y_test, model_name):
    """
    Avalia um modelo de regressão usando várias métricas.
    
    Args:
        model: Modelo de regressão treinado
        X_train: Features do conjunto de treino
        X_test: Features do conjunto de teste
        y_train: Target do conjunto de treino
        y_test: Target do conjunto de teste
        model_name: Nome do modelo para exibição
    
    Returns:
        dict: Dicionário com as métricas de avaliação
    """
    # Fazer predições
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Calcular métricas para o conjunto de treino
    train_r2 = r2_score(y_train, y_train_pred)
    train_mae = mean_absolute_error(y_train, y_train_pred)
    train_mse = mean_squared_error(y_train, y_train_pred)
    train_rmse = np.sqrt(train_mse)
    
    # Calcular métricas para o conjunto de teste
    test_r2 = r2_score(y_test, y_test_pred)
    test_mae = mean_absolute_error(y_test, y_test_pred)
    test_mse = mean_squared_error(y_test, y_test_pred)
    test_rmse = np.sqrt(test_mse)
    
    # Exibir resultados
    print(f"\nResultados para {model_name}:")
    print("\nMétricas no conjunto de treino:")
    print(f"R² Score: {train_r2:.4f}")
    print(f"MAE: {train_mae:.4f}")
    print(f"MSE: {train_mse:.4f}")
    print(f"RMSE: {train_rmse:.4f}")
    
    print("\nMétricas no conjunto de teste:")
    print(f"R² Score: {test_r2:.4f}")
    print(f"MAE: {test_mae:.4f}")
    print(f"MSE: {test_mse:.4f}")
    print(f"RMSE: {test_rmse:.4f}")
    
    # Visualizar predições vs valores reais
    plt.figure(figsize=(12, 5))
    
    # Gráfico para o conjunto de treino
    plt.subplot(1, 2, 1)
    plt.scatter(y_train, y_train_pred, alpha=0.5)
    plt.plot([y_train.min(), y_train.max()], [y_train.min(), y_train.max()], 'r--')
    plt.title(f'Valores Reais vs. Previsões (Treino)
R² = {train_r2:.4f}')
    plt.xlabel('Valores Reais')
    plt.ylabel('Previsões')
    
    # Gráfico para o conjunto de teste
    plt.subplot(1, 2, 2)
    plt.scatter(y_test, y_test_pred, alpha=0.5)
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
    plt.title(f'Valores Reais vs. Previsões (Teste)
R² = {test_r2:.4f}')
    plt.xlabel('Valores Reais')
    plt.ylabel('Previsões')
    
    plt.tight_layout()
    plt.savefig(os.path.join(IMAGES_DIR, f'model_predictions_{model_name.replace(" ", "_").lower()}.png'))
    plt.show()
    
    # Retornar métricas em um dicionário
    return {
        'model_name': model_name,
        'train_r2': train_r2,
        'train_mae': train_mae,
        'train_mse': train_mse,
        'train_rmse': train_rmse,
        'test_r2': test_r2,
        'test_mae': test_mae,
        'test_mse': test_mse,
        'test_rmse': test_rmse
    }

In [ ]:
def plot_feature_importance(model, feature_names, model_name):
    """
    Plota a importância das features para um modelo.
    
    Args:
        model: Modelo treinado
        feature_names: Lista com os nomes das features
        model_name: Nome do modelo para exibição
    
    Returns:
        DataFrame: DataFrame com a importância das features
    """
    # Verificar se o modelo tem atributo de importância de features
    if hasattr(model, 'feature_importances_'):
        # Para modelos baseados em árvores
        importances = model.feature_importances_
    elif hasattr(model, 'coef_'):
        # Para modelos lineares
        coefficients = model.coef_
        
        # Criar DataFrame para facilitar a ordenação
        importances = np.abs(coefficients)  # Usar valor absoluto para importância
    else:
        print(f"O modelo {model_name} não suporta visualização de importância de features.")
        return None
    
    # Criar DataFrame com importâncias
    importance_df = pd.DataFrame({
        'Feature': feature_names,
        'Importance': importances
    })
    
    # Ordenar por importância
    importance_df = importance_df.sort_values('Importance', ascending=False).reset_index(drop=True)
    
    # Plotar
    plt.figure(figsize=(10, 6))
    sns.barplot(x='Importance', y='Feature', data=importance_df)
    plt.title(f'Importância das Features - {model_name}')
    plt.xlabel('Importância')
    plt.ylabel('Feature')
    plt.tight_layout()
    plt.savefig(os.path.join(IMAGES_DIR, f'feature_importance_{model_name.replace(" ", "_").lower()}.png'))
    plt.show()
    
    # Exibir tabela de importância
    print(f"\nImportância das Features - {model_name}:")
    display(importance_df)
    
    return importance_df

### 6.3 Implementação e Avaliação dos Modelos

#### 6.3.1 Regressão Linear

In [ ]:
# Implementar e avaliar o modelo de Regressão Linear
print("\n" + "="*50)
print("MODELO 1: REGRESSÃO LINEAR")
print("="*50)

# Criar e treinar o modelo
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# Avaliar o modelo
lr_metrics = evaluate_regression_model(lr_model, X_train, X_test, y_train, y_test, "Regressão Linear")

# Visualizar importância das features
lr_importance = plot_feature_importance(lr_model, model_features, "Regressão Linear")

#### 6.3.2 Árvore de Decisão

In [ ]:
# Implementar e avaliar o modelo de Árvore de Decisão
print("\n" + "="*50)
print("MODELO 2: ÁRVORE DE DECISÃO")
print("="*50)

# Criar e treinar o modelo
dt_model = DecisionTreeRegressor(random_state=42)
dt_model.fit(X_train, y_train)

# Avaliar o modelo
dt_metrics = evaluate_regression_model(dt_model, X_train, X_test, y_train, y_test, "Árvore de Decisão")

# Visualizar importância das features
dt_importance = plot_feature_importance(dt_model, model_features, "Árvore de Decisão")

#### 6.3.3 Random Forest

In [ ]:
# Implementar e avaliar o modelo Random Forest
print("\n" + "="*50)
print("MODELO 3: RANDOM FOREST")
print("="*50)

# Criar e treinar o modelo
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Avaliar o modelo
rf_metrics = evaluate_regression_model(rf_model, X_train, X_test, y_train, y_test, "Random Forest")

# Visualizar importância das features
rf_importance = plot_feature_importance(rf_model, model_features, "Random Forest")

#### 6.3.4 Gradient Boosting (XGBoost)

In [ ]:
# Implementar e avaliar o modelo XGBoost
print("\n" + "="*50)
print("MODELO 4: XGBOOST")
print("="*50)

# Criar e treinar o modelo
xgb_model = XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
xgb_model.fit(X_train, y_train)

# Avaliar o modelo
xgb_metrics = evaluate_regression_model(xgb_model, X_train, X_test, y_train, y_test, "XGBoost")

# Visualizar importância das features
xgb_importance = plot_feature_importance(xgb_model, model_features, "XGBoost")

#### 6.3.5 Rede Neural (MLP Regressor)

In [ ]:
# Implementar e avaliar o modelo de Rede Neural
print("\n" + "="*50)
print("MODELO 5: REDE NEURAL (MLP REGRESSOR)")
print("="*50)

# Criar e treinar o modelo
mlp_model = MLPRegressor(hidden_layer_sizes=(100, 50), activation='relu', 
                        max_iter=1000, random_state=42)
mlp_model.fit(X_train, y_train)

# Avaliar o modelo
mlp_metrics = evaluate_regression_model(mlp_model, X_train, X_test, y_train, y_test, "MLP Regressor")

# O MLP não tem atributo de importância de features, então não podemos visualizar
print("\nNota: O modelo MLP Regressor não suporta visualização de importância de features.")

### 6.4 Comparação dos Modelos

In [ ]:
def compare_models(metrics_list):
    """
    Compara os modelos com base nas métricas de avaliação.
    
    Args:
        metrics_list (list): Lista de dicionários com métricas de cada modelo
    """
    print("\n" + "="*50)
    print("COMPARAÇÃO DOS MODELOS")
    print("="*50)
    
    # Criar DataFrame com as métricas de cada modelo
    comparison_df = pd.DataFrame(metrics_list)
    comparison_df = comparison_df.set_index('model_name')
    
    # Exibir tabela de comparação
    print("\nTabela de Comparação de Métricas:")
    display(comparison_df)
    
    # Identificar o melhor modelo com base no R² do conjunto de teste
    best_model_idx = comparison_df['test_r2'].idxmax()
    best_r2 = comparison_df.loc[best_model_idx, 'test_r2']
    print(f"\nMelhor modelo (R² no teste): {best_model_idx} (R² = {best_r2:.4f})")
    
    # Identificar o melhor modelo com base no RMSE do conjunto de teste
    best_rmse_idx = comparison_df['test_rmse'].idxmin()
    best_rmse = comparison_df.loc[best_rmse_idx, 'test_rmse']
    print(f"Melhor modelo (RMSE no teste): {best_rmse_idx} (RMSE = {best_rmse:.4f})")
    
    # Visualizar comparação de R²
    plt.figure(figsize=(12, 6))
    
    # Extrair nomes dos modelos e valores de R²
    model_names = comparison_df.index
    train_r2 = comparison_df['train_r2']
    test_r2 = comparison_df['test_r2']
    
    # Configurar barras
    x = np.arange(len(model_names))
    width = 0.35
    
    # Plotar barras
    plt.bar(x - width/2, train_r2, width, label='Treino')
    plt.bar(x + width/2, test_r2, width, label='Teste')
    
    # Adicionar rótulos e título
    plt.xlabel('Modelo')
    plt.ylabel('R² Score')
    plt.title('Comparação de R² Score entre Modelos')
    plt.xticks(x, model_names, rotation=45)
    plt.ylim(0, 1)  # R² geralmente varia de 0 a 1
    plt.legend()
    plt.grid(axis='y', alpha=0.3)
    plt.tight_layout()
    plt.savefig(os.path.join(IMAGES_DIR, 'model_r2_comparison.png'))
    plt.show()
    
    # Visualizar comparação de RMSE
    plt.figure(figsize=(12, 6))
    
    # Extrair valores de RMSE
    train_rmse = comparison_df['train_rmse']
    test_rmse = comparison_df['test_rmse']
    
    # Plotar barras
    plt.bar(x - width/2, train_rmse, width, label='Treino')
    plt.bar(x + width/2, test_rmse, width, label='Teste')
    
    # Adicionar rótulos e título
    plt.xlabel('Modelo')
    plt.ylabel('RMSE')
    plt.title('Comparação de RMSE entre Modelos')
    plt.xticks(x, model_names, rotation=45)
    plt.legend()
    plt.grid(axis='y', alpha=0.3)
    plt.tight_layout()
    plt.savefig(os.path.join(IMAGES_DIR, 'model_rmse_comparison.png'))
    plt.show()
    
    # Verificar overfitting
    print("\nVerificação de Overfitting:")
    for model in model_names:
        train_test_diff = comparison_df.loc[model, 'train_r2'] - comparison_df.loc[model, 'test_r2']
        if train_test_diff > 0.1:
            print(f"- {model}: Possível overfitting (diferença de R² entre treino e teste: {train_test_diff:.4f})")
        else:
            print(f"- {model}: Sem sinais claros de overfitting (diferença de R²: {train_test_diff:.4f})")
    
    # Conclusão e recomendações
    print("\nCONCLUSÃO:")
    print(f"Com base nas métricas de avaliação, o modelo {best_model_idx} apresentou o melhor desempenho geral.")
    
    # Verificar se o melhor modelo tem overfitting
    best_model_diff = comparison_df.loc[best_model_idx, 'train_r2'] - comparison_df.loc[best_model_idx, 'test_r2']
    if best_model_diff > 0.1:
        print(f"No entanto, este modelo apresenta sinais de overfitting. Recomenda-se ajustar hiperparâmetros ou usar técnicas de regularização.")
    
    return comparison_df

# Comparar todos os modelos
all_metrics = [lr_metrics, dt_metrics, rf_metrics, xgb_metrics, mlp_metrics]
model_comparison = compare_models(all_metrics)

## 7. Conclusões e Próximos Passos

### 7.1 Resumo dos Resultados

Neste projeto, realizamos uma análise completa de dados de produtividade agrícola, incluindo:

1. **Análise Exploratória de Dados**:
   - Identificamos as principais características do conjunto de dados
   - Analisamos a distribuição das variáveis
   - Verificamos a presença de valores ausentes e outliers

2. **Análise de Correlação**:
   - Identificamos as variáveis mais correlacionadas com a produtividade
   - Visualizamos as relações entre as variáveis através de matrizes de correlação e pairplots

3. **Análise de Clustering**:
   - Aplicamos o algoritmo K-Means para identificar grupos naturais nos dados
   - Determinamos o número ótimo de clusters usando o método do cotovelo e score de silhueta
   - Visualizamos os clusters em 2D e 3D
   - Identificamos outliers dentro dos clusters

4. **Modelagem Preditiva**:
   - Implementamos cinco modelos de aprendizado supervisionado
   - Avaliamos o desempenho dos modelos usando métricas como R², MAE, MSE e RMSE
   - Identificamos o modelo com melhor desempenho para prever a produtividade

Os resultados obtidos fornecem insights valiosos sobre os fatores que influenciam a produtividade agrícola e como podemos prever a produtividade com base nesses fatores.

### 7.2 Próximos Passos

Com base nos resultados obtidos, sugerimos os seguintes próximos passos:

1. **Otimização de Hiperparâmetros**:
   - Realizar uma busca mais exaustiva de hiperparâmetros para o modelo com melhor desempenho
   - Utilizar técnicas como Grid Search ou Random Search para encontrar a configuração ótima

2. **Feature Engineering**:
   - Criar novas features que possam capturar relações não lineares entre as variáveis
   - Explorar transformações de variáveis para melhorar o desempenho dos modelos

3. **Validação Cruzada**:
   - Implementar validação cruzada para obter uma avaliação mais robusta dos modelos
   - Testar a estabilidade dos modelos em diferentes subconjuntos dos dados

4. **Explicação do Modelo**:
   - Utilizar técnicas como SHAP (SHapley Additive exPlanations) para interpretar as predições do modelo
   - Fornecer explicações claras sobre como cada variável influencia a produtividade

5. **Implementação Prática**:
   - Desenvolver uma aplicação ou dashboard para que agricultores possam utilizar o modelo para prever a produtividade
   - Integrar o modelo com sistemas de monitoramento agrícola para fornecer recomendações em tempo real

Estes próximos passos permitirão aprimorar ainda mais a análise e tornar os resultados mais úteis para aplicações práticas no setor agrícola.